In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType,DoubleType,BooleanType,DateType 

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://tokyo-olampic-datafinal@tokyolampicdata.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)

In [0]:
import base64

# Replace these with your actual values
storage_account_name = "tokyolampicdata"
container_name = "tokyo-olampic-datafinal"
storage_account_key = "Enter your azure secret storage key"

# Validate the storage account key
def is_base64(s):
    try:
        return base64.b64encode(base64.b64decode(s)).decode() == s
    except Exception:
        return False

if not is_base64(storage_account_key):
    print("Error: The storage account key is not a valid base64-encoded string.")
    print("Please make sure you've copied the entire key from the Azure portal.")
else:
    print("Storage account key appears to be valid base64.")

    # Configure the connection
    spark.conf.set(
        f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
        storage_account_key
    )

    # Test the connection by trying to list files
    try:
        file_list = dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")
        print("Successfully connected to the storage account. Here are the first few files/directories:")
        for file in file_list[:5]:  # Print first 5 items
            print(file.name)
    except Exception as e:
        print(f"An error occurred while trying to list files: {str(e)}")

    # Try to read a small amount of data
    try:
        df = spark.read.text(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")
        print(f"Successfully read data. Row count: {df.count()}")
        print("First few rows:")
        df.show(5, truncate=False)
    except Exception as e:
        print(f"An error occurred while trying to read data: {str(e)}")

Storage account key appears to be valid base64.
Successfully connected to the storage account. Here are the first few files/directories:
raw-data/
transformed-data/
Successfully read data. Row count: 0
First few rows:
+-----+
|value|
+-----+
+-----+



In [0]:
# Replace these with your actual values
storage_account_name = "tokyolampicdata"
container_name = "tokyo-olampic-datafinal"
storage_account_key = "Enter your azure secret storage key"  

# Define the mount point
mount_point = "/mnt/tokyoolympic"

# Unmount if already mounted (optional, but helps prevent errors if re-running)
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)
    print(f"Unmounted existing mount point at {mount_point}")

# Configure the storage account access
configs = {
  f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key
}

# Perform the mount
try:
    dbutils.fs.mount(
        source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
        mount_point = mount_point,
        extra_configs = configs
    )
    print(f"Successfully mounted {container_name} to {mount_point}")
    
    # List contents of the mounted directory
    print("Contents of the mounted directory:")
    dbutils.fs.ls(mount_point)
except Exception as e:
    print(f"An error occurred while trying to mount: {str(e)}")

# Verify the mount
print("\nCurrent mounts:")
display(dbutils.fs.mounts())

# Try to access data in the mounted directory
try:
    df = spark.read.text(f"{mount_point}/raw-data")  # Adjust path if needed
    print(f"\nSuccessfully read data from {mount_point}/raw-data")
    print(f"Row count: {df.count()}")
    print("First few rows:")
    df.show(5, truncate=False)
except Exception as e:
    print(f"An error occurred while trying to read data from the mount: {str(e)}")

Successfully mounted tokyo-olampic-datafinal to /mnt/tokyoolympic
Contents of the mounted directory:

Current mounts:


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/mnt/tokyoolympic,wasbs://tokyo-olampic-datafinal@tokyolampicdata.blob.core.windows.net/,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,



Successfully read data from /mnt/tokyoolympic/raw-data
Row count: 12366
First few rows:
+-------------------------------------+
|value                                |
+-------------------------------------+
|PersonName,Country,Discipline        |
|AALERUD Katrine,Norway,Cycling Road  |
|ABAD Nestor,Spain,Artistic Gymnastics|
|ABAGNALE Giovanni,Italy,Rowing       |
|ABALDE Alberto,Spain,Basketball      |
+-------------------------------------+
only showing top 5 rows



In [0]:
%fs
ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,0
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,0


In [0]:
spark

In [0]:
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

# Initialize Spark session (this is usually already available in Databricks notebooks)
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

# Define the DBFS path where your CSV files are located
dbfs_path = "dbfs:/mnt/tokyoolympic/raw-data/"

# List files in the directory
file_list = dbutils.fs.ls(dbfs_path)

# Filter for CSV files
csv_files = [f.name for f in file_list if f.name.endswith('.csv')]

if not csv_files:
    print(f"No CSV files found in {dbfs_path}")
else:
    # Dictionary to store DataFrames
    dataframes = {}

    # Loop through each CSV file and read it into a DataFrame
    for csv_file in csv_files:
        # Get the file name without extension to use as the DataFrame name
        df_name = csv_file.rsplit('.', 1)[0]
        
        # Read the CSV file
        df = spark.read.format("csv") \
            .option("header", "True") \
                .option("inferSchema","true")\
            .load(dbfs_path + csv_file)
        
        # Store the DataFrame in the dictionary
        dataframes[df_name] = df

    # Print the names of all loaded DataFrames
    print("\nLoaded DataFrames:")
    for df_name in dataframes:
        print(df_name)

    # Show the first few rows of each DataFrame
    for df_name, df in dataframes.items():
        print(f"\nFirst few rows of '{df_name}' DataFrame:")
        df.show(5)

# Example of how to use the loaded DataFrames
if 'athletes' in dataframes:
    print("\nNumber of rows in athletes DataFrame:", dataframes['athletes'].count())


Loaded DataFrames:
athletes
coaches
entriesgender
medals
teams

First few rows of 'athletes' DataFrame:
+-----------------+-------+-------------------+
|       PersonName|Country|         Discipline|
+-----------------+-------+-------------------+
|  AALERUD Katrine| Norway|       Cycling Road|
|      ABAD Nestor|  Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|  Italy|             Rowing|
|   ABALDE Alberto|  Spain|         Basketball|
|    ABALDE Tamara|  Spain|         Basketball|
+-----------------+-------+-------------------+
only showing top 5 rows


First few rows of 'coaches' DataFrame:
+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|C�te d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+-----

In [0]:
athletes.show()


+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
entriesgender=entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType())) 

In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals

top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|                Cuba|   7|
|         New Zealand|   7|
|              Canada|   7|
|              Brazil|   7|
|   Republic of Korea|   6|
|             Hungary|   6|
|              Poland|   4|
|             Jamaica|   4|
|               Kenya|   4|
|      Czech Republic|   4|
|              Norway|   4|
|United States of ...|  39|
|People's Republic...|  38|
|              Sweden|   3|
|               Spain|   3|
|             Denmark|   3|
|         Switzerland|   3|
|             Croatia|   3|
|              Serbia|   3|
|Islamic Republic ...|   3|
+--------------------+----+
only showing top 20 rows



In [0]:
# Calculate the average number of entries by gender for each discipline

average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("dbfs:/mnt/tokyoolympic/transformed-data/athletes")


In [0]:

coaches.repartition(1).write.mode("overwrite").option("header","true").csv("dbfs:/mnt/tokyoolympic/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("dbfs:/mnt/tokyoolympic/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("dbfs:/mnt/tokyoolympic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("dbfs:/mnt/tokyoolympic/transformed-data/teams")